Dear tutor, there is something wrong with my IDE. I can change the file locally but cannot save my solution.
I also tried to upload my code to my personal Github repository but still cannot save the local change.
Although I could save the result of my spark code.

So I will put my code in this file. If you copy them to the original "hw07.ipynb" file and run, it should be fine.
All of the solution could get the right result.
Thanks so much. I am so sorry for this problem.



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# when run locally, spark has one (master) node with its own jvm and no cluster manager is created
spark = SparkSession.builder.master("local").appName("Homework 07").getOrCreate()

spark

In [ ]:
# load table from TPCxx-BB dataset. Returning a dataframe read from parquet format
get_table = lambda table: spark.read.option("header", "true").parquet(
    f"TPCx-BB-dataset/{table}.ptxt"
)

## Query 07
List top 10 states in descending order with at least 10 customers who during a given month bought products with the price tag at least 20% higher than the average price of products in the same category.

In [ ]:
# implementation

item = get_table("item")

# Define high price ratio
q07_HIGHER_PRICE_RATIO = 1.2

# Calculate the average price for each category and calculate the high price threshold
category_avg_price = item.groupBy("i_category").agg(
    (avg("i_current_price") * q07_HIGHER_PRICE_RATIO).alias("avg_price")
)

# Rename columns to avoid column name conflicts when joining
item = item.withColumnRenamed("i_category", "item_category")
item = item.withColumnRenamed("i_current_price", "item_price")

# Connect DataFrame and filter out high-priced items
high_price_items = (item.join(category_avg_price, item.item_category == category_avg_price.i_category)
    .filter(item.item_price > category_avg_price.avg_price)
    .select(item.i_item_sk)
)

customer_address = get_table("customer_address")
customer = get_table("customer")
store_sales = get_table("store_sales")
date_dim = get_table("date_dim")

q07_YEAR = 2004
q07_MONTH = 7
q07_HAVING_COUNT_GE = 10
q07_LIMIT = 10

# Extract all dates of a specific month from date_dim
selected_dates = date_dim.filter(
    (date_dim.d_year == q07_YEAR) & (date_dim.d_moy == q07_MONTH)
).select("d_date_sk").rdd.map(lambda row: row[0]).collect()

query7_solution = (customer_address
    .join(customer, customer_address.ca_address_sk == customer.c_current_addr_sk)
    .join(store_sales, customer.c_customer_sk == store_sales.ss_customer_sk)
    .join(high_price_items, store_sales.ss_item_sk == high_price_items.i_item_sk)
    .filter(store_sales.ss_sold_date_sk.isin(selected_dates))
    .filter(customer_address.ca_state.isNotNull())
    .groupBy(customer_address.ca_state)
    .agg(count("*").alias("cnt"))
    .filter(col("cnt") >= q07_HAVING_COUNT_GE)
    .orderBy(col("cnt"), ascending=False)
    .limit(q07_LIMIT)
)

query7_solution.show()
